In [ ]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data Overview**

We have gathered two key datasets to support our research and analysis. Below is a detailed breakdown of each dataset:

## 1. CCRB Dataset

### Dataset Source
- **Origin:** Civilian Complaint Review Board (CCRB)

### Timeframe of Data
- **Period:** 1985 - 2020

### Volume of Data
- **Total Complaints:** 33,358

### Dataset Composition
- The CCRB dataset is provided as a primary `.csv` file, along with an Excel file named `CCRB Data Layout Table`, which serves to clarify the terminology used within the dataset.

## 2. Additional Geospatial Data

This dataset complements the first by providing geospatial information crucial for in-depth analysis and visualization.

### Dataset Composition
- A shapefile dataset (`*.shp`) is included to facilitate precinct geolocation analysis, enhancing the exploratory data analysis (EDA) process.

### Accessibility
- The dataset, along with the `.csv` file and the Excel terminology guide, is accessible through a provided link.
- Precinct location data is derived from an exported shapefile (`.shp`), allowing for precise geographical mapping and analysis.

### Utilization
- This geospatial dataset is pivotal for uncovering patterns and trends within the civilian complaints filed against NYPD officers. It enables a spatial analysis approach, augmenting the predictive modeling of outcomes and fostering a deeper understanding of the impact of precinct locations on complaint frequencies and types.



In [ ]:
# navigating to the dataset folder uploaded to google drive
%cd "/content/drive/MyDrive/PoliceComplaintDataset"
!pwd

/content/drive/MyDrive/PoliceComplaintDataset
/content/drive/MyDrive/PoliceComplaintDataset


In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
import random
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import geopandas as gpd
from matplotlib import pyplot
from geopy.geocoders import Nominatim
import datetime

In [ ]:
#importing the dataset
df=pd.read_csv("allegations_202007271729.csv")
df.shape

(33358, 27)

In [ ]:
df.head()

,unique_mos_id,first_name,last_name,command_now,shield_no,complaint_id,month_received,year_received,month_closed,year_closed,...,mos_age_incident,complainant_ethnicity,complainant_gender,complainant_age_incident,fado_type,allegation,precinct,contact_reason,outcome_description,board_disposition
0,10004,Jonathan,Ruiz,078 PCT,8409,42835,7,2019,5,2020,...,32,Black,Female,38.0,Abuse of Authority,Failure to provide RTKA card,78.0,Report-domestic dispute,No arrest made or summons issued,Substantiated (Command Lvl Instructions)
1,10007,John,Sears,078 PCT,5952,24601,11,2011,8,2012,...,24,Black,Male,26.0,Discourtesy,Action,67.0,Moving violation,Moving violation summons issued,Substantiated (Charges)
2,10007,John,Sears,078 PCT,5952,24601,11,2011,8,2012,...,24,Black,Male,26.0,Offensive Language,Race,67.0,Moving violation,Moving violation summons issued,Substantiated (Charges)
3,10007,John,Sears,078 PCT,5952,26146,7,2012,9,2013,...,25,Black,Male,45.0,Abuse of Authority,Question,67.0,PD suspected C/V of violation/crime - street,No arrest made or summons issued,Substantiated (Charges)
4,10009,Noemi,Sierra,078 PCT,24058,40253,8,2018,2,2019,...,39,NaN,NaN,16.0,Force,Physical force,67.0,Report-dispute,Arrest - other violation/crime,Substantiated (Command Discipline A)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33358 entries, 0 to 33357
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   unique_mos_id             33358 non-null  int64  
 1   first_name                33358 non-null  object 
 2   last_name                 33358 non-null  object 
 3   command_now               33358 non-null  object 
 4   shield_no                 33358 non-null  int64  
 5   complaint_id              33358 non-null  int64  
 6   month_received            33358 non-null  int64  
 7   year_received             33358 non-null  int64  
 8   month_closed              33358 non-null  int64  
 9   year_closed               33358 non-null  int64  
 10  command_at_incident       31814 non-null  object 
 11  rank_abbrev_incident      33358 non-null  object 
 12  rank_abbrev_now           33358 non-null  object 
 13  rank_now                  33358 non-null  object 
 14  rank_i

In [ ]:
df.describe()

,unique_mos_id,shield_no,complaint_id,month_received,year_received,month_closed,year_closed,mos_age_incident,complainant_age_incident,precinct
count,33358.000000,33358.000000,33358.000000,33358.000000,33358.000000,33358.000000,33358.000000,33358.000000,28546.000000,33334.000000
mean,18169.912495,6451.608819,23905.058217,6.323551,2010.726782,6.470772,2011.525661,32.346873,32.484201,64.365213
std,9566.316896,7945.641596,11954.434138,3.362951,6.034725,3.343372,6.085907,6.040944,28.408963,31.451592
min,2.000000,0.000000,517.000000,1.000000,1985.000000,1.000000,1985.000000,20.000000,-4301.000000,0.000000
25%,9671.000000,1089.000000,13684.750000,3.000000,2007.000000,4.000000,2008.000000,28.000000,23.000000,43.000000
50%,19215.000000,3691.000000,25132.000000,6.000000,2012.000000,6.000000,2013.000000,31.000000,30.000000,67.000000
75%,25412.000000,7155.000000,34252.000000,9.000000,2016.000000,9.000000,2016.000000,36.000000,41.000000,81.000000
max,36374.000000,31977.000000,43703.000000,12.000000,2020.000000,12.000000,2020.000000,60.000000,101.000000,1000.000000


In [ ]:
df.columns

Index(['unique_mos_id', 'first_name', 'last_name', 'command_now', 'shield_no',
       'complaint_id', 'month_received', 'year_received', 'month_closed',
       'year_closed', 'command_at_incident', 'rank_abbrev_incident',
       'rank_abbrev_now', 'rank_now', 'rank_incident', 'mos_ethnicity',
       'mos_gender', 'mos_age_incident', 'complainant_ethnicity',
       'complainant_gender', 'complainant_age_incident', 'fado_type',
       'allegation', 'precinct', 'contact_reason', 'outcome_description',
       'board_disposition'],
      dtype='object')

In [ ]:
df.isna().sum()

unique_mos_id                  0
first_name                     0
last_name                      0
command_now                    0
shield_no                      0
complaint_id                   0
month_received                 0
year_received                  0
month_closed                   0
year_closed                    0
command_at_incident         1544
rank_abbrev_incident           0
rank_abbrev_now                0
rank_now                       0
rank_incident                  0
mos_ethnicity                  0
mos_gender                     0
mos_age_incident               0
complainant_ethnicity       4464
complainant_gender          4195
complainant_age_incident    4812
fado_type                      0
allegation                     1
precinct                      24
contact_reason               199
outcome_description           56
board_disposition              0
dtype: int64

# **Merging base dataset with latitude and longitude dataset**

In [ ]:
location= gpd.read_file("geo_export_359d6e65-21a5-4212-a07f-e715385fc5bc.shp")
location = location.to_crs(4326)
location['longitude'] = location.centroid.x
location['latitude'] = location.centroid.y
df=df.merge(location, how='right', on='precinct')

<ipython-input-11-2ee30262bf62>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location['longitude'] = location.centroid.x
<ipython-input-11-2ee30262bf62>:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  location['latitude'] = location.centroid.y


# **Adding an additional column and binning from data received and date closed**

In [ ]:
df['date_received'] = pd.to_datetime(df['year_received'].astype(str) + '-' +
                                     df['month_received'].astype(str) + '-01')
df['date_closed'] = pd.to_datetime(df['year_closed'].astype(str) + '-' +
                                   df['month_closed'].astype(str) + '-01')
df['days_taken_to_resolve']=(df['date_closed']-df['date_received']).dt.days
df['bins_day']=np.floor(df['days_taken_to_resolve']/30)

In [ ]:
df[['days_taken_to_resolve','bins_day']]

,days_taken_to_resolve,bins_day
0,305,10.0
1,305,10.0
2,151,5.0
3,151,5.0
4,183,6.0
...,...,...
33305,122,4.0
33306,153,5.0
33307,181,6.0
33308,334,11.0


In [ ]:
df.columns

Index(['unique_mos_id', 'first_name', 'last_name', 'command_now', 'shield_no',
       'complaint_id', 'month_received', 'year_received', 'month_closed',
       'year_closed', 'command_at_incident', 'rank_abbrev_incident',
       'rank_abbrev_now', 'rank_now', 'rank_incident', 'mos_ethnicity',
       'mos_gender', 'mos_age_incident', 'complainant_ethnicity',
       'complainant_gender', 'complainant_age_incident', 'fado_type',
       'allegation', 'precinct', 'contact_reason', 'outcome_description',
       'board_disposition', 'shape_area', 'shape_leng', 'geometry',
       'longitude', 'latitude', 'date_received', 'date_closed',
       'days_taken_to_resolve', 'bins_day'],
      dtype='object')

# **Preparing new dataset for EDA after merging with the location dataset**

In [ ]:
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderInsufficientPrivileges

graph_data = df.groupby(['precinct']).size().reset_index(name='frequency')
merged_data = location.merge(graph_data, how='right', on='precinct')
merged_data['latitude']=merged_data['latitude'].apply(str)
merged_data['longitude']=merged_data['longitude'].apply(str)
geolocator = Nominatim(user_agent="geoapiExercises")



In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderUnavailable, GeocoderTimedOut, GeocoderServiceError
import time

geolocator = Nominatim(user_agent="nypd", timeout=10)

def zipcode_of_location(row):
    location_str = f"{row['latitude']},{row['longitude']}"
    retries = 3
    for attempt in range(retries):
        try:
            location = geolocator.reverse(location_str)
            zipcode = location.raw.get('address', {}).get('postcode', '')
            return zipcode
        except (GeocoderTimedOut, GeocoderUnavailable) as e:
            print(f"Retrying... Attempt {attempt + 1} of {retries}")
            time.sleep(1)
        except Exception as e:
            print(f"Error: {e}")
            return None
    return None

merged_data['zipcode'] = merged_data.apply(zipcode_of_location, axis=1)
merged_data_sorted = merged_data.sort_values('frequency', ascending=False)
merged_data_sorted = merged_data_sorted.head()

In [ ]:
merged_data_sorted

,precinct,shape_area,shape_leng,geometry,longitude,latitude,frequency,zipcode
46,75.0,1.804941e+08,106337.879200,"MULTIPOLYGON (((-73.85023 40.62332, -73.85023 ...",-73.87775660857048,40.665009662366174,2172,11208
45,73.0,5.175437e+07,33000.209960,"POLYGON ((-73.90405 40.67922, -73.90349 40.678...",-73.91094655789068,40.6681587790829,1163,11212
26,44.0,5.551179e+07,31464.134787,"POLYGON ((-73.90269 40.84458, -73.90271 40.844...",-73.9207952796168,40.83380611153094,1139,10452
28,46.0,3.833342e+07,29392.712842,"POLYGON ((-73.89138 40.86170, -73.89142 40.861...",-73.90957642771251,40.852746440383804,1120,10453
39,67.0,9.381497e+07,43255.869527,"POLYGON ((-73.90755 40.65119, -73.90849 40.650...",-73.93381756632431,40.64804204405852,1119,11203


# **Abbreviations**

In [ ]:
xls = pd.ExcelFile('CCRB Data Layout Table.xlsx')
df_ranks = pd.read_excel(xls, 'Rank Abbrevs')
df_command = pd.read_excel(xls, 'Command Abbrevs')

In [ ]:
rank_dict=dict(zip(df_ranks.Abbreviation,df_ranks.Rank))
command_dict=dict(zip(df_command.Abbreviation,df_command['Command Name']))

In [ ]:
df['command_now'].value_counts()

command_now
INT CIS    1412
WARRSEC    1158
NARCBBX     692
DB CEIS     681
075 PCT     415
           ... 
C A B         1
CD IRS        1
PB HRD        1
C/PRV D       1
TRN BUR       1
Name: count, Length: 415, dtype: int64

In [ ]:
df['rank_abbrev_incident'].value_counts()

rank_abbrev_incident
POM    19799
SGT     5698
DT3     2697
POF     1397
PO      1288
LT      1262
DTS      329
DT2      193
CPT      182
SDS      128
SSA      105
DI        96
DET       50
INS       27
LSA       24
DT1       20
LCD       13
DC         2
Name: count, dtype: int64

In [ ]:
# Mapping the abbreviations to command names in two columns of a dataframe
df['command_now'] = df['command_now'].map(command_dict)
df['command_at_incident'] = df['command_at_incident'].map(command_dict)

# Mapping the abbreviations to command names in two columns of a dataframe
df['rank_abbrev_incident'] = df['rank_abbrev_incident'].map(rank_dict)
df['rank_abbrev_now'] = df['rank_abbrev_now'].map(rank_dict)

In [ ]:
df['command_now'].value_counts()

command_now
Criminal Intelligence Section                       1412
Warrant Section                                     1158
Narcotics Borough Bronx                              692
Organized Crime Investigation Division               681
075 Precinct                                         415
                                                    ... 
Facilities Management Division                         2
Chief of Department Investigation Review Section       1
Emergency Operations Center                            1
Crime Prevention Division                              1
Community Affairs Bureau                               1
Name: count, Length: 385, dtype: int64

In [ ]:
df['rank_abbrev_incident'].value_counts()

rank_abbrev_incident
Police Officer                    22484
Sergeant                           5698
Detective Grade 3                  2697
Lieutenant                         1262
Detective Specialist                329
Detective Grade 2                   193
Captain                             182
Sergeant Detective Squad            128
Sergeant Special Assignment         105
Deputy Inspector                     96
Inspector                            27
Lieutenant Special Assignment        24
Detective Grade 1                    20
Lieutenant Commander Detective       13
Deputy Chief                          2
Name: count, dtype: int64

# **Dropping Columns**

In [ ]:
df.drop(['rank_now','rank_incident','shield_no'],axis=1,inplace=True)

# **Removing NAs with number of rows < 200**

In [ ]:
df.dropna(subset=['allegation','precinct','contact_reason','outcome_description'],axis=0,inplace=True,how='any')

# **Combining some categories and filling NAs with mean, percentile_random_value, and mode value**

In [ ]:
df["complainant_ethnicity"]=np.where(
    ((df["complainant_ethnicity"]=="Unknown") | (df["complainant_ethnicity"]=="Other Race") | (df["complainant_ethnicity"]=="Refused")),
    "Not disclosed",
    df["complainant_ethnicity"]
)
df.complainant_ethnicity.value_counts()

complainant_ethnicity
Black              17011
Hispanic            6384
White               2754
Not disclosed       1957
Asian                527
American Indian       63
Name: count, dtype: int64

In [ ]:
df['complainant_gender'].value_counts()

complainant_gender
Male                     23904
Female                    4969
Not described               57
Transwoman (MTF)            20
Transman (FTM)               5
Gender non-conforming        2
Name: count, dtype: int64

In [ ]:
df['complainant_gender']=np.where(
    df['complainant_gender'].isna(),
    df['complainant_gender'].value_counts().index[0],
    df['complainant_gender']
)
df['complainant_gender'].value_counts()

complainant_gender
Male                     28005
Female                    4969
Not described               57
Transwoman (MTF)            20
Transman (FTM)               5
Gender non-conforming        2
Name: count, dtype: int64

In [ ]:
df['command_at_incident'].value_counts()

command_at_incident
075 Precinct                                 1360
046 Precinct                                  824
044 Precinct                                  754
Narcotics Borough Brooklyn North              746
079 Precinct                                  735
                                             ... 
Applicant Processing Division                   1
048 Detective Squad                             1
060 Precinct                                    1
Detective Borough Queens South Operations       1
122 Detective Squad                             1
Name: count, Length: 271, dtype: int64

In [ ]:
top5_commands=df['command_at_incident'].value_counts(normalize=True)[0:5].index.to_list()
top5_commands_weights=df['command_at_incident'].value_counts(normalize=True)[0:5].to_list()
top5_commands_weights=list(normalize([top5_commands_weights],norm="l1")[0])
print(top5_commands)
print(top5_commands_weights)

['075 Precinct', '046 Precinct', '044 Precinct', 'Narcotics Borough Brooklyn North', '079 Precinct']
[0.30776193708983934, 0.18646752658972618, 0.17062683865127856, 0.16881647431545596, 0.16632722335369993]


In [ ]:
randomSampleList = np.random.choice(top5_commands,df['command_at_incident'].isna().sum(),replace=True,p=top5_commands_weights)
df['command_at_incident'].fillna(pd.Series(randomSampleList,index=df['command_at_incident'].index
                                           [df['command_at_incident'].isna()]), inplace=True)

In [ ]:
df['command_at_incident'].value_counts()

command_at_incident
075 Precinct                        2222
046 Precinct                        1422
044 Precinct                        1278
079 Precinct                        1257
Narcotics Borough Brooklyn North    1239
                                    ... 
Emergency Service Squad 06             1
084 Detective Squad                    1
OMAP                                   1
Gang Division                          1
122 Detective Squad                    1
Name: count, Length: 271, dtype: int64

# **Combining column and dropping original columns**

In [ ]:
df['name']=df['first_name']+" "+df['last_name']
df.drop(['first_name','last_name'],axis=1,inplace=True)

In [ ]:
df.head()

,unique_mos_id,command_now,complaint_id,month_received,year_received,month_closed,year_closed,command_at_incident,rank_abbrev_incident,rank_abbrev_now,...,shape_area,shape_leng,geometry,longitude,latitude,date_received,date_closed,days_taken_to_resolve,bins_day,name
0,10134,079 Precinct,24367,10,2011,8,2012,001 Precinct,Police Officer,Lieutenant,...,4.728642e+07,80283.538778,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ...",-74.012069,40.709791,2011-10-01,2012-08-01,305,10.0,James Hart
1,10134,079 Precinct,24367,10,2011,8,2012,001 Precinct,Police Officer,Lieutenant,...,4.728642e+07,80283.538778,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ...",-74.012069,40.709791,2011-10-01,2012-08-01,305,10.0,James Hart
2,10134,079 Precinct,26826,10,2012,3,2013,001 Precinct,Police Officer,Lieutenant,...,4.728642e+07,80283.538778,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ...",-74.012069,40.709791,2012-10-01,2013-03-01,151,5.0,James Hart
3,10134,079 Precinct,26826,10,2012,3,2013,001 Precinct,Police Officer,Lieutenant,...,4.728642e+07,80283.538778,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ...",-74.012069,40.709791,2012-10-01,2013-03-01,151,5.0,James Hart
4,10134,079 Precinct,32585,4,2015,10,2015,001 Precinct,Police Officer,Lieutenant,...,4.728642e+07,80283.538778,"MULTIPOLYGON (((-74.04388 40.69019, -74.04351 ...",-74.012069,40.709791,2015-04-01,2015-10-01,183,6.0,James Hart


In [ ]:
print(df.describe())

       unique_mos_id  complaint_id  month_received  year_received  \
count   33058.000000  33058.000000    33058.000000   33058.000000   
mean    18166.338466  23933.525380        6.322554    2010.749440   
min         2.000000    517.000000        1.000000    1985.000000   
25%      9671.000000  13788.000000        3.000000    2007.000000   
50%     19211.500000  25142.000000        6.000000    2012.000000   
75%     25412.000000  34222.000000        9.000000    2016.000000   
max     36374.000000  43703.000000       12.000000    2020.000000   
std      9564.189515  11898.461739        3.360411       5.997266   

       month_closed   year_closed  mos_age_incident  complainant_age_incident  \
count  33058.000000  33058.000000      33058.000000              28342.000000   
mean       6.475195   2011.548067         32.350596                 32.467433   
min        1.000000   1985.000000         20.000000              -4301.000000   
25%        4.000000   2008.000000         28.000000   

In [ ]:
ethnicity_contact_crosstab = pd.crosstab(df['contact_reason'], df['complainant_ethnicity'])
ethnicity_contact_crosstab

complainant_ethnicity,American Indian,Asian,Black,Hispanic,Not disclosed,White
contact_reason,,,,,,
Aided case,0,4,65,26,10,26
Arrest/Complainant,0,0,2,0,0,0
Assist ACS or other agency,0,0,27,0,1,0
C/V at PCT to file complaint of crime,0,0,46,28,7,16
C/V at PCT to obtain information,0,3,85,35,14,24
C/V at PCT to retrieve property,0,3,15,10,2,6
C/V intervened on behalf of/observed encounter w/3rd party,0,15,272,124,48,52
C/V requested info from officer,0,0,45,19,23,19
C/V requested investigation of crime,2,7,133,54,31,66


In [ ]:
ethnicity_outcome_crosstab = pd.crosstab(df['outcome_description'], df['complainant_ethnicity'])
ethnicity_outcome_crosstab

complainant_ethnicity,American Indian,Asian,Black,Hispanic,Not disclosed,White
outcome_description,,,,,,
Arrest - OGA,0,12,420,100,24,38
Arrest - assault (against a PO),0,8,546,139,46,75
Arrest - disorderly conduct,0,7,592,249,41,70
Arrest - harrassment (against a PO),0,0,8,0,3,1
Arrest - other violation/crime,8,121,5379,2128,629,668
Arrest - resisting arrest,0,25,888,373,95,111
Juvenile Report,0,0,35,6,15,0
Moving violation summons issued,2,54,389,149,50,159
No arrest made or summons issued,33,215,6072,2174,764,1164


In [ ]:
rank_contact_crosstab = pd.crosstab(df['contact_reason'], df['rank_abbrev_incident'])
rank_contact_crosstab

rank_abbrev_incident,Captain,Deputy Chief,Deputy Inspector,Detective Grade 1,Detective Grade 2,Detective Grade 3,Detective Specialist,Inspector,Lieutenant,Lieutenant Commander Detective,Lieutenant Special Assignment,Police Officer,Sergeant,Sergeant Detective Squad,Sergeant Special Assignment
contact_reason,,,,,,,,,,,,,,,
Aided Cases,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0
Aided case,0,0,0,0,0,0,4,0,7,0,0,130,13,0,0
Arrest/Complainant,0,0,0,0,0,0,0,0,2,0,0,66,3,0,0
Arrest/Not Complainant,0,0,0,0,0,0,0,0,0,0,0,30,2,0,0
Assist ACS or other agency,0,0,0,0,1,3,0,0,0,0,0,19,13,0,3
C/V at PCT to file complaint of crime,5,0,0,0,0,9,0,0,0,0,0,57,30,0,1
C/V at PCT to obtain information,1,0,1,0,0,3,1,0,12,0,0,98,47,0,2
C/V at PCT to retrieve property,1,0,0,0,0,4,0,0,1,0,0,20,13,0,0
C/V intervened on behalf of/observed encounter w/3rd party,1,0,0,0,0,18,3,0,24,0,0,421,73,0,0
